# Prepare Illinois Energy Plant Data

In [ ]:
# imports
import pandas as pd
import re

In [ ]:
import re

def convertDMSToDD(degrees: str, minutes: str, seconds: str, direction: str):
	dd = float(degrees) + (float(minutes)/60) + (float(seconds)/(60*60))

	if direction == "S" or direction == "W":
		dd *= -1

	return dd

def castDMS(input: str):
	"""
	cast input from coord to decimal

	old regex: /\d+(\.\d+)?°(\d+(\.\d+)?′)?(\d+(\.\d+)?″)?[NESW]/
	"""
	parts = re.split("[^\d\w\.]+", input)

	if re.match("\d+(\.\d+)?°\d+(\.\d+)?′\d+(\.\d+)?″[NESW]", input):
		# 38°16′40.2″N
		return convertDMSToDD(parts[0], parts[1], parts[2], parts[3])
	elif re.match("\d+(\.\d+)?°\d+(\.\d+)?′[NESW]", input):
		# 38.204°N
		return convertDMSToDD(parts[0], parts[1], 0, parts[2])
	elif re.match("\d+(\.\d+)?°[NESW]", input):
		# 38°16.12N
		return convertDMSToDD(parts[0], 0, 0, parts[1])
	else:
		# it's something else, just return original input
		return input

In [ ]:
csv_path = "../data/energy/illinois/natural_gas_plants.tsv"
df = pd.read_csv(csv_path, delimiter="\t")

if 'coordinates' in df:
    df['yCoordinate'] = df.apply(lambda row: castDMS(row.coordinates.split(' ')[0]), axis=1)
    df['xCoordinate'] = df.apply(lambda row: castDMS(row.coordinates.split(' ')[1]), axis=1)
    df.to_csv(csv_path, sep="\t", index=False)
else:
    print("'coordinates' column not found in the file")